***Color manual***

<div class="alert alert-block alert-success">
<b>Green - Libraries</b>
</div>

<div class="alert alert-block alert-info">
<b>Blue - Information</b>
</div>

<div class="alert alert-block alert-danger">
<b>Red - Problems</b>
</div>

***

<div class="alert alert-block alert-success">
<b>Import libraries</b>
</div>

In [1]:
import numpy as np
import chart_studio
import pandas as pd
import plotly.graph_objs as go
import chart_studio.plotly as py

chart_studio.tools.set_config_file(world_readable = True, sharing = 'public')

# Change default pandas display options
pd.options.display.float_format = '{:,.2f}'.format

<div class="alert alert-block alert-info">
<b>Load the Data, set index and info about data</b>
</div>

In [2]:
df = pd.read_excel('Cestas.xlsx', index_col = 'Alimento', sheet_name = 'Data frame')

df.head()

,Faixa etária,Quantidade de alunos,Quantidade/aluno,kg/unidade,Preço/unidade,kg/aluno,Preço/aluno,kg total,Preço total
Alimento,,,,,,,,,
Açucar,3 anos a 6 anos,2158,1,1.00,2.29,1.00,2.29,"2,158.00","4,941.82"
Açucar,6 anos a 15 anos,9691,1,1.00,2.29,1.00,2.29,"9,691.00","22,192.39"
Arroz,6 meses a 3 anos,1998,1,1.00,2.89,1.00,2.89,"1,998.00","5,774.22"
Arroz,3 anos a 6 anos,2158,2,1.00,2.89,2.00,5.78,"4,316.00","12,473.24"
Arroz,6 anos a 15 anos,9691,2,1.00,2.89,2.00,5.78,"19,382.00","56,013.98"


<div class="alert alert-block alert-info">
<b>Pivot table sorted</b>
</div>

In [3]:
pivot_all = pd.pivot_table(df, index = ['Alimento','Faixa etária'], 
                               values = ['Quantidade/aluno', 'kg total', 'Preço total'],
                               aggfunc = np.sum, fill_value = 0)


# Select data that will show and change the index order
pivot_all = pivot_all.reindex([('Arroz', '6 meses a 3 anos'),
                               ('Arroz',  '3 anos a 6 anos'),
                               ('Arroz', '6 anos a 15 anos'),
                               ('Feijão', '6 meses a 3 anos'),
                               ('Feijão',  '3 anos a 6 anos'),
                               ('Feijão', '6 anos a 15 anos')], axis = 0)

# Sort by alimento
pivot_all = pivot_all.sort_values(by = ['Alimento'], ascending = True)

# Reorder the values of pivot table
pivot_all = pivot_all[['Quantidade/aluno', 'kg total', 'Preço total']]

pivot_all.head(9)

Quantidade/aluno  kg total  Preço total
Alimento Faixa etária                                             
Arroz    6 meses a 3 anos                 1  1,998.00     5,774.22
         3 anos a 6 anos                  2  4,316.00    12,473.24
         6 anos a 15 anos                 2 19,382.00    56,013.98
Feijão   6 meses a 3 anos                 1  1,998.00    12,567.42
         3 anos a 6 anos                  1  2,158.00    13,573.82
         6 anos a 15 anos                 1  9,691.00    60,956.39

<div class="alert alert-block alert-info">
<b>Pivot table MultiIndex sorted</b>
</div>

In [4]:
pivot_one = pd.pivot_table(df, index = ['Alimento'],
                               columns = ['Faixa etária'],
                               values = ['kg total', 'Preço total'],
                               aggfunc = np.sum, fill_value = 0)

# Change the columns order
pivot_one = pivot_one.reindex([('Preço total', '6 meses a 3 anos'), 
                               ('Preço total', '3 anos a 6 anos'),
                               ('Preço total', '6 anos a 15 anos'),
                               ('kg total', '6 meses a 3 anos'), 
                               ('kg total', '3 anos a 6 anos'),
                               ('kg total', '6 anos a 15 anos')], axis = 1)

# Sort by preço total with 6 meses a 3 anos
pivot_one = pivot_one.sort_values(by = [('Preço total', '6 meses a 3 anos')], ascending = False)

# Reorder the values of pivot table
pivot_one = pivot_one[['kg total', 'Preço total']]

pivot_one

kg total                                   \
Faixa etária        6 meses a 3 anos 3 anos a 6 anos 6 anos a 15 anos   
Alimento                                                                
Leite                       1,598.40        1,726.40             0.00   
Mucilon                       919.08            0.00             0.00   
Feijão                      1,998.00        2,158.00         9,691.00   
Arroz                       1,998.00        4,316.00        19,382.00   
Macarrão                      999.00        2,158.00         9,691.00   
Açucar                          0.00        2,158.00         9,691.00   
Biscoito cracker                0.00          863.20         3,876.40   
Biscoito maria                  0.00          863.20         3,876.40   
Charque traseiro                0.00            0.00         4,845.50   
Farinha de mandioca             0.00            0.00         9,691.00   
Massa de milho                  0.00        2,158.00         9,691.00   
Sal                             0.00        2,158.00         9,691.00   
Óleo                            0.00        1,942.20         8,721.90   

                         Preço total                                   
Faixa etária        6 meses a 3 anos 3 anos a 6 anos 6 anos a 15 anos  
Alimento                                                               
Leite                      37,482.48       40,484.08             0.00  
Mucilon                    19,540.44            0.00             0.00  
Feijão                     12,567.42       13,573.82        60,956.39  
Arroz                       5,774.22       12,473.24        56,013.98  
Macarrão                    3,976.02        8,588.84        38,570.18  
Açucar                          0.00        4,941.82        22,192.39  
Biscoito cracker                0.00        9,689.42        43,512.59  
Biscoito maria                  0.00       12,710.62        57,079.99  
Charque traseiro                0.00            0.00       119,102.39  
Farinha de mandioca             0.00            0.00        28,976.09  
Massa de milho                  0.00        6,430.84        28,879.18  
Sal                             0.00        1,704.82         7,655.89  
Óleo                            0.00        7,747.22        34,790.69

<div class="alert alert-block alert-info">
<b>Data frame to make plot</b>
</div>

In [5]:
df_ini = pd.read_excel('Cestas.xlsx', sheet_name = '6 meses a 3 anos')
df_ini.set_index('alimento', inplace = True)
df_ini.sort_index(inplace = True)

df_mei = pd.read_excel('Cestas.xlsx', sheet_name = '3 anos a 6 anos')
df_mei.set_index('alimento', inplace = True)
df_mei.sort_index(inplace = True)

df_fin = pd.read_excel('Cestas.xlsx', sheet_name = '6 anos a 15 anos')
df_fin.set_index('alimento', inplace = True)  
df_fin.sort_index(inplace = True)

# Example of format of the three data frames for each group
df_ini

,quantidade,kg/unidade,preço/unidade,kg/aluno,preço/aluno,kg total,preço total
alimento,,,,,,,
Arroz,1,1.00,2.89,1.00,2.89,"1,998.00","5,774.22"
Feijão,1,1.00,6.29,1.00,6.29,"1,998.00","12,567.42"
Leite,4,0.20,4.69,0.80,18.76,"1,598.40","37,482.48"
Macarrão,1,0.50,1.99,0.50,1.99,999.00,"3,976.02"
Mucilon,2,0.23,4.89,0.46,9.78,919.08,"19,540.44"


<div class="alert alert-block alert-info">
<b>Bar plot</b>
</div>

In [6]:
fig = go.Figure()

fig.add_trace(go.Bar(x = df_ini.index, y = df_ini['preço total'], 
    name = '6 meses a 3 anos',
    hoverinfo = 'x+y+text',
    marker_color = 'rgb(255, 255, 145)',
    marker_line_color = 'rgb(1,1,1)',
    marker_line_width = 2))

fig.add_trace(go.Bar(x = df_mei.index, y = df_mei['preço total'], 
    name = '3 anos a 6 anos', 
    hoverinfo = 'x+y+text',
    marker_color = 'rgb(115, 150, 200)',
    marker_line_color = 'rgb(1,1,1)',
    marker_line_width = 2))

fig.add_trace(go.Bar(x = df_fin.index, y = df_fin['preço total'],
    name = '6 anos a 15 anos',
    hoverinfo = 'x+y+text',
    marker_color ='rgb(190, 230, 160)',
    marker_line_color = 'rgb(1,1,1)',
    marker_line_width = 2))

fig.update_layout(barmode = 'stack', 
    title_text = 'Cestas por grupos', 
    title_x = 0.5,
    title_y = 0.9,
    legend = dict(x = 0, y = 1.5),
    legend_title='<b> Grupos </b>',
    xaxis_title = 'Alimentos',
    yaxis_title = 'Preço total',
    paper_bgcolor = 'rgba(1,1,1,1)',
    plot_bgcolor = 'rgba(1,1,1,1)',
    font = dict(
    family = 'Helvetica, Helvetica',
    size = 15,
    color = 'white'))

fig.update_yaxes(hoverformat = '.2f', showgrid = False)

py.iplot(fig, filename = 'Cestas por grupos', auto_open = False)